In [1]:
import time

start_time = time.time()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
1,application_1543381571657_0002,pyspark3,idle,Link,Link,✔


SparkSession available as 'spark'.


In [1]:
import subprocess

# Install H2O PySparkling
stdout = subprocess.check_output(
    "pip3 install h2o_pysparkling_2.3",
    stderr=subprocess.STDOUT,
    shell=True).decode("utf-8")
print(stdout)


  Running setup.py bdist_wheel for h2o-pysparkling-2.3: started
  Running setup.py bdist_wheel for h2o-pysparkling-2.3: finished with status 'done'
  Stored in directory: /home/.cache/pip/wheels/0c/cd/e6/65d3a3c52525dc1ea3d13ad17e0935172db2f0dcd4a08322ba
  Running setup.py bdist_wheel for tabulate: started
  Running setup.py bdist_wheel for tabulate: finished with status 'done'
  Stored in directory: /home/.cache/pip/wheels/2a/85/33/2f6da85d5f10614cbe5a625eab3b3aebfdf43e7b857f25f829
  Running setup.py bdist_wheel for future: started
  Running setup.py bdist_wheel for future: finished with status 'done'
  Stored in directory: /home/.cache/pip/wheels/0c/61/d2/d6b7317325828fbb39ee6ad559dbe4664d0896da4721bf379e
  Running setup.py bdist_wheel for pyspark: started
  Running setup.py bdist_wheel for pyspark: finished with status 'done'
  Stored in directory: /home/.cache/pip/wheels/be/7d/34/cd3cfbc75d8b6b6ae0658e5425348560b86d187fe3e53832cc
Successfully built h2o-pysparkling-2.3 tabulate futu

In [1]:
dataFileName = "powerplant_output.csv"
dataFileUrl = "https://raw.githubusercontent.com/h2oai/h2o-tutorials/master/h2o-world-2017/automl/data/" + dataFileName

# Download data file and copy to HDFS, if not already there
cmd = 'hdfs dfs -ls /tmp/' + dataFileName + ' || ' \
    '(wget ' + dataFileUrl + ' && ' \
    'hdfs dfs -copyFromLocal ' + dataFileName + ' /tmp && ' \
    'rm ' + dataFileName + ')'

stdout = subprocess.check_output(
    cmd,
    stderr=subprocess.STDOUT,
    shell=True).decode("utf-8")
print(stdout)

ls: `/tmp/powerplant_output.csv': No such file or directory
--2018-11-28 18:21:03--  https://raw.githubusercontent.com/h2oai/h2o-tutorials/master/h2o-world-2017/automl/data/powerplant_output.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.48.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.48.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 308777 (302K) [text/plain]
Saving to: 'powerplant_output.csv'

     0K .......... .......... .......... .......... .......... 16%  293K 1s
    50K .......... .......... .......... .......... .......... 33%  568K 1s
   100K .......... .......... .......... .......... .......... 49%  294K 0s
   150K .......... .......... .......... .......... .......... 66%  228K 0s
   200K .......... .......... .......... .......... .......... 82%  193K 0s
   250K .......... .......... .......... .......... .......... 99%  231K 0s
   300K .                                      

In [1]:
from pysparkling import H2OContext

hc = H2OContext.getOrCreate(spark)

Connecting to H2O server at http://10.244.0.66:54323... successful.
--------------------------  ---------------------------------------------------
H2O cluster uptime:         12 secs
H2O cluster timezone:       Etc/UTC
H2O data parsing timezone:  UTC
H2O cluster version:        3.22.0.2
H2O cluster version age:    6 days
H2O cluster name:           sparkling-water-root_application_1543381571657_0002
H2O cluster total nodes:    3
H2O cluster free memory:    3.698 Gb
H2O cluster total cores:    48
H2O cluster allowed cores:  3
H2O cluster status:         accepting new members, healthy
H2O connection url:         http://10.244.0.66:54323
H2O connection proxy:
H2O internal security:      False
H2O API Extensions:         XGBoost, Algos, AutoML, Core V3, Core V4
Python version:             3.5.2 final
--------------------------  ---------------------------------------------------

Sparkling Water Context:
 * H2O name: sparkling-water-root_application_1543381571657_0002
 * cluster size: 3
 

In [1]:
# Print the hostname
stdout = subprocess.check_output(
    "hostname",
    stderr=subprocess.STDOUT,
    shell=True).decode("utf-8")
print(stdout)

mssql-storage-pool-default-0

In [1]:
from pysparkling.ml import H2OAutoML
from pyspark.ml import Pipeline
from pyspark.ml.feature import SQLTransformer

powerplant_df = spark.read.option("inferSchema", "true").csv("/tmp/powerplant_output.csv", header=True)

splits = powerplant_df.randomSplit([0.8, 0.2], seed=1)
train = splits[0]
for_predictions = splits[1]

temperatureTransformer = SQLTransformer(statement="SELECT * FROM __THIS__ WHERE TemperatureCelcius > 10")

automlEstimator = H2OAutoML(maxModels=2, predictionCol="HourlyEnergyOutputMW",
    ratio=0.9, seed=1)

pipeline = Pipeline(stages=[temperatureTransformer, automlEstimator])
model = pipeline.fit(train)
predicted = model.transform(for_predictions)

predicted.show()


+------------------+---------------+-----------------------+----------------+--------------------+--------------------+
|TemperatureCelcius|ExhaustVacuumHg|AmbientPressureMillibar|RelativeHumidity|HourlyEnergyOutputMW|   prediction_output|
+------------------+---------------+-----------------------+----------------+--------------------+--------------------+
|             10.01|          41.17|                1018.78|           86.84|               479.4|[478.98443473089264]|
|             10.02|          39.66|                1016.34|           79.98|              480.05| [478.4744401166399]|
|             10.03|          43.13|                1014.85|           70.09|              482.16|[475.98085258410845]|
|             10.04|          41.62|                1013.36|           95.17|              463.87|[468.36092209033416]|
|             10.05|          41.58|                1021.35|           95.19|              469.03|[468.67240194913506]|
|             10.06|          34.69|    

In [1]:
automlEstimator.leaderboard().show(truncate=False)

+---------------------------------------------------+----------------------+------------------+------------------+------------------+---------------------+
|model_id                                           |mean_residual_deviance|rmse              |mse               |mae               |rmsle                |
+---------------------------------------------------+----------------------+------------------+------------------+------------------+---------------------+
|StackedEnsemble_BestOfFamily_AutoML_20181128_003911|11.00711037433692     |3.317696546451607 |11.00711037433692 |2.460978645004018 |0.0073362304526039795|
|StackedEnsemble_AllModels_AutoML_20181128_003911   |11.00711037433692     |3.317696546451607 |11.00711037433692 |2.460978645004018 |0.0073362304526039795|
|XRT_1_AutoML_20181128_003911                       |11.18040320029184     |3.3437109923394757|11.18040320029184 |2.5010207855031896|0.007393792671247257 |
|DRF_1_AutoML_20181128_003911                       |11.24517840

In [1]:
from pyspark.sql.functions import *

scores = predicted.select(predicted['HourlyEnergyOutputMW'], predicted['prediction_output']['value'].alias('prediction'))


In [1]:
from pyspark.ml.evaluation import RegressionEvaluator

evaluator = RegressionEvaluator(predictionCol="prediction",
                                labelCol="HourlyEnergyOutputMW",
                                metricName="mae")

mae = evaluator.evaluate(scores)

print("Mean absolute error:", mae)

Mean absolute error: 2.4058999799874976

In [1]:
from pyspark.ml.evaluation import RegressionEvaluator

evaluator = RegressionEvaluator(predictionCol="prediction",
                                labelCol="HourlyEnergyOutputMW",
                                metricName="r2")

r2 = evaluator.evaluate(scores)

print("r squared:", r2)

r squared: 0.9516895664346249

In [1]:
elapsed_time = time.time() - start_time
print("Elapsed time: {:.1f} seconds.".format(elapsed_time))

Elapsed time: 199.5 seconds.